In [ ]:
# Install required packages (uncomment to run)
# !pip install langchain==0.3.19 langchain-community==0.3.18 openai==1.65.2 ipykernel==6.29.5


In [ ]:
# -------------------- Libraries and Modules -------------
from langchain import PromptTemplate, LLMChain
from langchain_community.chat_models.perplexity import ChatPerplexity
from langchain.chains import SequentialChain
import os


In [ ]:
# -------------------- Initialization --------------------
# IMPORTANT: Do NOT hard-code API keys in source. Set the PPLX_API_KEY in your environment or GitHub Secrets.
PPLX_API_KEY = os.environ.get("PPLX_API_KEY")  # set this in your environment (export PPLX_API_KEY=...)

# Initialize the Perplexity/Chat model using the environment-provided key
llm = ChatPerplexity(model="sonar-reasoning", 
                     temperature=0.5, 
                     pplx_api_key=PPLX_API_KEY)
# -------------------- Chain Definitions --------------------
market_data_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["ticker"],
        template=(
            "📈 For {ticker}, provide detailed and up-to-date financial data. Include current stock price, "
            "volume, key financial ratios (e.g., P/E, P/B, dividend yield), recent price trends, and relevant market indicators."
        )
    ),
    output_key="market_data"
)

sentiment_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["ticker"],
        template=(
            "📰 For {ticker}, analyze recent news articles, social media posts, and expert commentary. "
            "Summarize the prevailing sentiment, highlight any key events, and note emerging trends that may impact the stock."
        )
    ),
    output_key="sentiment_analysis"
)

macro_analysis_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["ticker"],
        template=(
            "🌐 For {ticker}, analyze the current macro-economic environment. "
            "Include key indicators such as GDP growth, inflation rates, interest rates, unemployment trends, "
            "and central bank policies. Summarize how these factors could impact the overall market and the asset."
        )
    ),
    output_key="macro_analysis"
)

strategy_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["market_data", "sentiment_analysis", "macro_analysis"],
        template=(
            "📊 Using the detailed market data:
{market_data}
"
            "the sentiment analysis:
{sentiment_analysis}
"
            "and the macro-economic analysis:
{macro_analysis}
"
            "develop a sophisticated trading strategy. Outline a clear asset allocation, specify entry and exit points, "
            "detail risk management measures, and provide estimated expected returns. If applicable, incorporate algorithmic signals."
        )
    ),
    output_key="strategy"
)

risk_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["strategy"],
        template=(
            "⚠️ Evaluate the following trading strategy:
{strategy}
"
            "Identify potential risks such as market volatility, liquidity issues, or unexpected market events. "
            "Summarize your risk assessment in 4 concise bullet points, and state in the final bullet point whether the strategy meets an acceptable risk tolerance."
        )
    ),
    output_key="risk_assessment"
)

sequential_agent = SequentialChain(
    chains=[market_data_chain, sentiment_chain, macro_analysis_chain, strategy_chain, risk_chain],
    input_variables=["ticker"],
    output_variables=["market_data", "sentiment_analysis", "macro_analysis", "strategy", "risk_assessment"],
    verbose=True
)

def run_ai_hedge_fund(ticker: str) -> None:
    result = sequential_agent({"ticker": ticker})
    
    print("
======== AI Hedge Fund Analysis Results ========
")
    
    print("📈 Market Data Retrieved:")
    print("--------------------------------------------------")
    print(result["market_data"], "
")
    
    print("📰 Market Sentiment Analysis:")
    print("--------------------------------------------------")
    print(result["sentiment_analysis"], "
")
    
    print("🌐 Macro-Economic Analysis:")
    print("--------------------------------------------------")
    print(result["macro_analysis"], "
")
    
    print("📊 Developed Trading Strategy:")
    print("--------------------------------------------------")
    print(result["strategy"], "
")
    
    print("⚠️ Risk Assessment:")
    print("--------------------------------------------------")
    print(result["risk_assessment"], "
")
    
    print("==============================================
")

# Uncomment to run in an environment with PPLX_API_KEY set
# run_ai_hedge_fund("S&P500")
